Proje: https://www.kaggle.com/competitions/platesv2/overview

In [2]:
import os
os.environ['KAGGLE_CONFIG_DIR'] = '/content'
!kaggle competitions download -c platesv2

 81% 38.0M/47.0M [00:00<00:00, 65.1MB/s]
100% 47.0M/47.0M [00:00<00:00, 64.9MB/s]


In [3]:
!unzip platesv2.zip

Archive:  platesv2.zip
  inflating: plates.zip              
  inflating: sample_submission.csv   


In [4]:
!unzip plates.zip

Archive:  plates.zip
   creating: plates/
  inflating: plates/.DS_Store        
   creating: __MACOSX/
   creating: __MACOSX/plates/
  inflating: __MACOSX/plates/._.DS_Store  
   creating: plates/test/
  inflating: plates/test/0071.jpg    
  inflating: plates/test/0717.jpg    
  inflating: plates/test/0703.jpg    
  inflating: plates/test/0065.jpg    
  inflating: plates/test/0059.jpg    
  inflating: plates/test/0515.jpg    
  inflating: plates/test/0273.jpg    
  inflating: plates/test/0267.jpg    
  inflating: plates/test/0501.jpg    
  inflating: plates/test/0529.jpg    
  inflating: plates/test/0298.jpg    
  inflating: plates/test/0461.jpg    
  inflating: plates/test/0307.jpg    
  inflating: plates/test/0313.jpg    
  inflating: plates/test/0475.jpg    
  inflating: plates/test/0449.jpg    
  inflating: plates/test/0105.jpg    
  inflating: plates/test/0663.jpg    
  inflating: plates/test/0677.jpg    
  inflating: plates/test/0111.jpg    
  inflating: plates/test/0139.jpg    


In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.applications import VGG16
from tensorflow.keras.optimizers import Adam

In [17]:
os.listdir('/content/plates/train')

['.DS_Store', 'dirty', 'cleaned']

In [16]:
train = '/content/plates/train'  # Eğitim veri seti yolu
test = '/content/plates'  # Test veri seti yolu

In [17]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator,img_to_array,load_img

In [18]:
validation_split = 0.2
train_datagen=ImageDataGenerator(validation_split=validation_split,rescale=1./255,rotation_range=45,shear_range=0.2,width_shift_range=0.2,height_shift_range=0.2,zoom_range=0.2,horizontal_flip=True,fill_mode='nearest')
validation_datagen=ImageDataGenerator(validation_split=validation_split ,rescale=1./255)
test_datagen=ImageDataGenerator(rescale=1./255)

In [20]:
train_generator=train_datagen.flow_from_directory(directory=train,classes=['cleaned','dirty'],batch_size=64,class_mode='binary',shuffle=True,target_size=(224,224),subset='training')
validation_generator=validation_datagen.flow_from_directory(directory=train,classes=['cleaned','dirty'],batch_size=64,class_mode='binary',shuffle=True,target_size=(224,224),subset='validation')
test_generator=test_datagen.flow_from_directory(directory=test,classes=['test'],batch_size = 1,shuffle = False,class_mode=None,target_size=(224,224))

Found 32 images belonging to 2 classes.
Found 8 images belonging to 2 classes.
Found 744 images belonging to 1 classes.


In [25]:
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,InputLayer,BatchNormalization,Dropout,Flatten
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

In [22]:
basemodel=VGG16(include_top=False,weights='imagenet',input_shape=(224,224,3))

In [23]:
# Modelin son katmanlarını dondurma
for layer in base_model.layers:
    layer.trainable = False

In [26]:
model=Sequential()
model.add(basemodel)
model.add(Flatten())
model.add(Dense(512,activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.01)))
model.add(Dropout(0.6))
model.add(BatchNormalization())
model.add(Dense(1,activation='sigmoid'))

In [27]:
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

In [28]:
earlystopping=EarlyStopping(monitor='val_loss',patience=3,restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=1e-6)

In [30]:
history=model.fit(train_generator,epochs=50,validation_data=validation_generator,callbacks=[earlystopping,reduce_lr],verbose=1)

Epoch 1/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 89s 89s/step - accuracy: 0.7500 - loss: 9.4303 - val_accuracy: 0.5000 - val_loss: 177.1857 - learning_rate: 0.0010
Epoch 2/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 87s 87s/step - accuracy: 0.5938 - loss: 8.5818 - val_accuracy: 0.5000 - val_loss: 54.2198 - learning_rate: 0.0010
Epoch 3/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 132s 132s/step - accuracy: 0.4375 - loss: 7.7041 - val_accuracy: 0.5000 - val_loss: 149.6128 - learning_rate: 0.0010
Epoch 4/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 74s 74s/step - accuracy: 0.7812 - loss: 6.5623 - val_accuracy: 0.5000 - val_loss: 337.8863 - learning_rate: 0.0010
Epoch 5/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 82s 82s/step - accuracy: 0.3125 - loss: 6.1618 - val_accuracy: 0.5000 - val_loss: 333.3261 - learning_rate: 0.0010


In [31]:
import glob
from sklearn.utils import shuffle
import matplotlib.pyplot as plt

In [32]:
IMG_DIM = (224, 224)
test = glob.glob('/content/plates/test/*.jpg')
test = shuffle(test)[:50]

In [34]:
# Extract images and labels
test_imgs = []
test_labels = []

for img in test:
    # Load and resize image
    img_array =img_to_array(load_img(img, target_size=IMG_DIM))
    test_imgs.append(img_array)

# Convert lists to numpy arrays
test_imgs = np.array(test_imgs)
test_labels = np.array(test_labels)

# Standardize
test_imgs_scaled = test_imgs.astype('float32') / 255.0

# Define functions for label conversion
def class2num(label):
    return 0 if label == 'cleaned' else 1

def num2class(num):
    return 'cleaned' if num < 0.5 else 'dirty'

# Convert labels to numerical format for model training
test_labels_enc = np.array([class2num(label) for label in test_labels])

# Predict using the model
predictions_model_history = model.predict(test_imgs_scaled, verbose=0)

# Convert predictions from probabilities to class labels
predictions_model_labels = np.array([num2class(pred[0]) for pred in predictions_model_history])

In [36]:
preds = model.predict(test_generator, verbose=True)
labels = ['dirty' if x > 0.5 else 'cleaned' for x in preds]
submission_df = pd.read_csv('/content/sample_submission.csv')
submission_df['label'] = labels

744/744 ━━━━━━━━━━━━━━━━━━━━ 473s 636ms/step


In [37]:
submission_df.to_csv('submission.csv', index=False)